# CSV EVENTS FILE Cleanup

## Deprecated

Run the python module `prepare_events_CSV.py` instead.

In [44]:
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [45]:
os.listdir('data')

['2022-03-13_eventsDB.csv',
 '2022-03-16_eventsDB.csv',
 '2022-03-17_eventsDB.csv',
 'adds1548_1554.csv',
 'backup_eventsDB_.js',
 'bkups',
 'celebrity_deaths_4.csv',
 'df_main.csv',
 'economists_lifetimes.csv',
 'eventsDB.js',
 'eventsDB.test',
 'eventsDB_.js',
 'eventsWIP.xlsx',
 'events_db.csv',
 'events_master.csv',
 'list_of_presidents_of_france-836j.csv',
 'list_of_prime_ministers_of_uk-839j.csv',
 'pantheon.tsv',
 'philosophers.csv',
 'README.md',
 'US_Presidents.json',
 'world_events.js',
 'world_events.json',
 'world_events.txt']

In [46]:
master_file = 'data/events_master.csv'
df = pd.read_csv(master_file, encoding = "ISO-8859-1")
df = df.replace(r'^\s+$', np.nan, regex=True) #get rid of unwanted blank spaces in cells...

In [47]:
df['event'] = df['event'].str.strip()
#df = df.drop(['Alternatives'], axis=1)

In [48]:
print(df.shape, df.columns)

(215, 28) Index(['eventID', 'stem', 'event', 'details', 'wikipedia', 'mnemonic',
       'itemDifficulty', 'categories', 'America', 'Europe', 'Asia', 'Africa',
       'China', 'Discovery', 'Britain', 'France', 'Greek', 'India',
       'Invention', 'MiddleEast', 'prehistory', 'Religion', 'Roman', 'Royalty',
       'Russia', 'Science', 'Wars', 'World'],
      dtype='object')


In [49]:
df.sample(3)

,eventID,stem,event,details,wikipedia,mnemonic,itemDifficulty,categories,America,Europe,...,Invention,MiddleEast,prehistory,Religion,Roman,Royalty,Russia,Science,Wars,World
119,119.0,1945,End of Second World War;,NaN,NaN,NaN,1.0,"[Wars, ]",NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63.0,1609,Kepler discovers that planets have elliptical ...,Kepler discovered that the orbit of Mars was a...,NaN,NaN,6.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
194,NaN,240 BC,Greek mathematician Eratosthenes (276194) mea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# might need to do this for other columns as well.
categories = ['Asia', 'China', 'Britain', 'Roman', 'America']
for c in categories:
    df = df.astype({c: 'float64'})

In [51]:
#verify
df.select_dtypes('object')

,stem,event,details,wikipedia,mnemonic,categories
0,c.3500 BC,Invention of the wheel and plough in Mesopotam...,NaN,NaN,NaN,"[prehistory,]"
1,c.3200 BC,Invention of writing in Mesopotamia: the means...,NaN,NaN,NaN,"[prehistory,]"
2,c.3000 BC,Founding of the first cities in Sumeria (prese...,NaN,NaN,NaN,"[prehistory,]"
3,c.3000 BC,Building of the Great Pyramid.,NaN,NaN,NaN,"[prehistory,]"
4,c.1600 BC,Modern alphabet invented: the essential means ...,NaN,NaN,NaN,"[prehistory,]"
...,...,...,...,...,...,...
210,753 BC,Legendary founding of Rome.,NaN,NaN,NaN,NaN
211,995 BC,The Judean King David captures Jerusalem.,NaN,NaN,NaN,NaN
212,1200 BC,Consensus for the Fall of Troy (if there was a...,NaN,NaN,NaN,NaN
213,1871,"""Dr. Livingstone, I presume"": US journalist St...",After nearly eight months Stanley found Living...,https://en.wikipedia.org/wiki/Henry_Morton_Sta...,"Slave trade ended ~1850, and colonization of C...",NaN


In [52]:
print(df.select_dtypes('object').shape)

(215, 6)


In [53]:
col = "America"
df[col].unique()
#cond = (df[col]=="1")
#df[cond]

array([nan,  1.])

In [1]:
df.describe().columns

NameError: name 'df' is not defined

In [55]:
df.sum()

eventID                                                       16764
stem              c.3500 BCc.3200 BCc.3000 BCc.3000 BCc.1600 BCc...
event             Invention of the wheel and plough in Mesopotam...
itemDifficulty                                                 1039
America                                                          25
Europe                                                           79
Asia                                                             32
Africa                                                            8
China                                                            14
Discovery                                                         6
Britain                                                          29
France                                                            8
Greek                                                            17
India                                                            14
Invention                                       

In [56]:
df.shape

(215, 28)

## Adding A Year Num Column

### 1. YearNum


In [57]:
def getYearNum(x):
    mult = 1
    if "BC" in x:
        x = x.replace('BC', '')                
        mult = -1
    x = x.replace('c.','') #handle circa
    x = x.replace('AD','')
        
    try:
        yr = mult * int(x)
    except:
        print('CONVERSION ERROR while attempting...')
        print(x)
        yr = x
        
    return(yr)

In [58]:
df['YearNum'] = df['stem'].apply(getYearNum)

In [59]:
df = df.sort_values(by='YearNum')

In [60]:
df['eventID'] = range(0, df.shape[0])

In [61]:
df

,eventID,stem,event,details,wikipedia,mnemonic,itemDifficulty,categories,America,Europe,...,MiddleEast,prehistory,Religion,Roman,Royalty,Russia,Science,Wars,World,YearNum
0,0,c.3500 BC,Invention of the wheel and plough in Mesopotam...,NaN,NaN,NaN,6.0,"[prehistory,]",NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3500
1,1,c.3200 BC,Invention of writing in Mesopotamia: the means...,NaN,NaN,NaN,8.0,"[prehistory,]",NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3200
2,2,c.3000 BC,Founding of the first cities in Sumeria (prese...,NaN,NaN,NaN,8.0,"[prehistory,]",NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3000
3,3,c.3000 BC,Building of the Great Pyramid.,NaN,NaN,NaN,4.0,"[prehistory,]",NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3000
4,4,c.1600 BC,Modern alphabet invented: the essential means ...,NaN,NaN,NaN,8.0,"[prehistory,]",NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,210,1996,Dolly the sheep becomes the first mammal to be...,Her birth proved that specialised cells could ...,https://en.wikipedia.org/wiki/Dolly_(sheep),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996
179,211,1997,Tony Blair back in power in UK. Mohd. Khatami ...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997
180,212,1998,Indonesian President Suharto resigns. Pakistan...,NaN,NaN,NaN,8.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998
181,213,1999,G-15 Summit ends,NaN,NaN,NaN,6.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1999


In [62]:
from datetime import date

today = date.today()
print(f"Today's date: {today}")


Today's date: 2022-04-04


In [63]:
csvfilename = f'data/{today}_events.csv'
df.to_csv(csvfilename, index=False)
print(f'wrote file {csvfilename}')

wrote file data/2022-04-04_events.csv


# END Here